# Summarization

## Зависимости

In [1]:
# !conda update -n base -c conda-forge conda -y
# !conda install -c conda-forge ipywidgets -y
# !conda install -c conda-forge datasets -y
# !conda install -c conda-forge transformers -y
# !conda install pytorch torchvision torchaudio pytorch-cuda=11.8 -c pytorch -c nvidia -y
# %pip install evaluate -qq
# %pip install rouge-score -qq
# %pip install nltk -qq
# %pip install absl-py -qq
# %pip install transformers[sentencepiece] -qq

## Подготовка данных

In [1]:
import random

import pandas as pd

from datasets import load_dataset, ClassLabel
from IPython.display import display, HTML

### csebuetnlp/xlsum

In [2]:
dataset_xlsum_ru = load_dataset("csebuetnlp/xlsum", 'russian')

In [3]:
dataset_xlsum_ru

DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'title', 'summary', 'text'],
        num_rows: 62243
    })
    test: Dataset({
        features: ['id', 'url', 'title', 'summary', 'text'],
        num_rows: 7780
    })
    validation: Dataset({
        features: ['id', 'url', 'title', 'summary', 'text'],
        num_rows: 7780
    })
})

In [4]:
def show_random_elements(dataset, num_examples=2):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [5]:
show_random_elements(dataset_xlsum_ru['train'])

## Обучение моделей

In [6]:
import gc
import os
import comet_ml

from accelerate import notebook_launcher
from accelerate.utils import set_seed

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

import torch
import torch.optim as optim

[2023-09-30 09:22:15,642] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [7]:
os.environ["COMET_LOG_ASSETS"] = "True"

### facebook bart-large-cnn

In [8]:
model_checkpoint = "facebook/bart-large-cnn"

#### Tokenizer

In [9]:
old_tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [10]:
tokenizer = old_tokenizer.train_new_from_iterator(dataset_xlsum_ru, 250_027)
tokenizer.save_pretrained("facebook/bart-large-cnn-ru")

('facebook/bart-large-cnn-ru/tokenizer_config.json',
 'facebook/bart-large-cnn-ru/special_tokens_map.json',
 'facebook/bart-large-cnn-ru/vocab.json',
 'facebook/bart-large-cnn-ru/merges.txt',
 'facebook/bart-large-cnn-ru/added_tokens.json',
 'facebook/bart-large-cnn-ru/tokenizer.json')

In [11]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [12]:
tokenized_datasets = dataset_xlsum_ru.map(tokenize_function, batched=True, num_proc=8)

In [13]:
# block_size = tokenizer.model_max_length
BLOCK_SIZE = 128
BATCH_SIZE = 10

In [14]:
prefix = "summarize: "

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["summary"], max_length=BLOCK_SIZE, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [15]:
lm_datasets = tokenized_datasets.map(
    preprocess_function,
    batched=True,
    batch_size=BATCH_SIZE,
    num_proc=8,
)

Map (num_proc=8):   0%|          | 0/62243 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/7780 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/7780 [00:00<?, ? examples/s]

In [16]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [17]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model_checkpoint)

#### GPU runtime

In [18]:
torch.cuda.empty_cache()
gc.collect()

92

In [19]:
!nvidia-smi

Sat Sep 30 09:23:49 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 520.61.05    Driver Version: 520.61.05    CUDA Version: 11.8     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  On   | 00000000:01:00.0 Off |                    0 |
| N/A   33C    P0    26W / 250W |      0MiB / 32768MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-PCIE...  On   | 00000000:02:00.0 Off |                    0 |
| N/A   

#### Evaluate

In [20]:
import evaluate

rouge = evaluate.load("rouge")

In [21]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

#### Training

In [22]:
comet_ml.init( project_name = "KaiTube", experiment_name = "Summarization")

In [23]:
def training_function():
    model_name = model_checkpoint.split("/")[-1]
    
    training_args = Seq2SeqTrainingArguments(
        output_dir="my_awesome_billsum_model",
        evaluation_strategy="epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=BATCH_SIZE,
        per_device_eval_batch_size=BATCH_SIZE,
        weight_decay=0.01,
        save_total_limit=3,
        num_train_epochs=20,
        predict_with_generate=True,
        fp16=True
    )
    
    set_seed(42)
    torch.manual_seed(7)
    
    optimizer = optim.AdamW(model.parameters(), training_args.learning_rate)
    scheduler = optim.lr_scheduler.OneCycleLR(
        optimizer, 
        max_lr=training_args.learning_rate,
        steps_per_epoch=int(len(lm_datasets["train"])),
        epochs=training_args.per_device_train_batch_size,
        anneal_strategy='linear'
    )
    
    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=lm_datasets["train"],
        eval_dataset=lm_datasets["validation"],
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
        optimizers=(optimizer, scheduler)
    )
    trainer.train()
    return trainer

In [24]:
trainer = notebook_launcher(training_function, num_processes=2, mixed_precision='fp16')

Launching training on 2 GPUs.


COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET WARNING: You are trying to log string value as a metric. This is not recommended.
COMET INFO: Experiment is live on comet.com https://www.comet.com/gumaonelove/kaitube/3303e3048d624240a389735f733e8fd8

You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
[W reducer.cpp:1300] Warning: find_unused_parameters=True was specified in DDP constructor, but did not find any unused parameters in the forward pass. This flag resul

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,1.076600,1.003059,0.004800,0.000600,0.004800,0.004800,140.472800
2,0.932100,0.866360,0.007000,0.000900,0.007000,0.007000,139.982500
3,0.842300,0.781446,0.007000,0.000800,0.007000,0.007000,134.843800


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,1.076600,1.003059,0.004800,0.000600,0.004800,0.004800,140.472800
2,0.932100,0.866360,0.007000,0.000900,0.007000,0.007000,139.982500
3,0.842300,0.781446,0.007000,0.000800,0.007000,0.007000,134.843800


KeyboardInterrupt: 

#### Inference

In [25]:
from transformers import pipeline

In [26]:
summarizer = pipeline("summarization", model="my_awesome_billsum_model/checkpoint-9500/")

In [27]:
text = """['Оживленная городская улица ночью с множеством огней.'] 
['Улыбающаяся женщина держит табличку с надписью «Я не могу видеть изображение выше».'] 
['Черно-белое фото человека, сидящего на полу.'] 
['Стол, на котором лежат разные предметы.'] 
['Стол, на котором лежат разные предметы.'] 
['Стол, на котором лежат разные предметы.'] 
['Женщина сидит за столом с ноутбуком.'] 
['Женщина, сидящая на скамейке, смотрит в мобильный телефон.'] 
['Женщина, сидящая в кресле, разговаривает по мобильному телефону.'] 
['Черно-белое фото человека, держащего мобильный телефон.'] 
['Черно-белая фотография человека, держащего мобильный телефон.'] 
['Две фотографии человека, держащего два разных предмета.'] 
['Стеклянная скульптура слона, сидящего на столе.'] 
['Стеклянная скульптура слона, сидящего на столе.'] 
['Черно-белая фотография человека, держащего табличку.'] 
['Черно-белая фотография человека, держащего мобильный телефон.'] 
['Крупный план трех разных изображений.'] 
['Две фотографии человека, держащего сноуборд.'] 
['Плюшевый мишка с розовым бантом.'] 
['Сине-белый уличный фонарь и дерево.'] 
['Стеклянная ваза с нарисованными на ней цветами.'] 
['Стеклянная ваза с нарисованным на ней лицом.'] 
['Стеклянная ваза стоит на столе.'] 
['Сине-белый уличный фонарь и дерево.'] 
['Крупный план часов на стене здания.'] 
['Крупный план цветка на фоне неба.'] 
['Черно-белая фотография человека, стоящего на коленях на земле.'] 
['Крупный план розового цветка в вазе.'] 
['Крупный план человека, держащего скейтборд.'] 
['Крупный план желтого и зеленого цветов.'] 
['Две доски для серфинга стоят рядом друг с другом.'] 
['Розовый плюшевый мишка сидит в вазе.'] 
['Крупный план уличного знака на фоне голубого неба.'] 
['Крупный план знака остановки на фоне неба.'] 
['Уличная сцена с изображением знака «Стоп».'] 
['Черно-белое фото трех человек, держащих плюшевых мишек.'] 
['Группа людей, стоящих рядом друг с другом.'] 
['Черно-белое фото человека, держащего мобильный телефон.'] 
['Крупный план человека, стоящего рядом с автомобилем.'] 
['Светофор с двумя дорожными знаками на нем.'] 
['Стеклянный шар, наполненный множеством разных вещей.'] 
['Группа людей, запускающих воздушных змеев в парке.'] 
['Группа людей, стоящих вокруг и держащих воздушных змеев.'] 
['Куча воздушных змеев, которые летают в воздухе.'] 
['Куча фотографий, на которых изображены разные предметы.'] 
['Стеклянная скульптура, сделанная в виде слона.'] 
['Черно-белое фото сноубордиста в шлеме'] 
['Светофор, на котором есть надписи.'] 
['Крупный план человека, запускающего воздушного змея.'] 
['Плюшевый мишка, который держит человек.'] 
['Крупный план человека, держащего скейтборд.'] 
['Крупный план человека, запускающего воздушного змея.'] 
['Крупный план часов на стене.'] 
['Крупный план цветка в вазе.'] 
['Светофор на фоне неба.'] 
['Крупный план уличного знака, сделанного в виде коровы.'] 
['Две фотографии человека, держащего сноуборд.'] 
['Крупный план трех разных предметов.'] 
['Уличная сцена с изображением знака остановки.'] 
['Улыбающаяся женщина держит табличку с надписью «С днем \u200b\u200bрождения».'] 
['Крупный план трех разных цветов.'] 
['Стеклянная ваза с изображением цветов.'] 
['Маленькая девочка сидит на стуле в комнате.'] 
['Скейтборд и другие предметы на столе.'] 
['Крупный план двух разных часов.'] 
['Стеклянный шар, наполненный множеством разных изображений.'] 
['Стеклянная скульптура, изображающая двух человек, держащих в руках предметы.'] 
['Куча воздушных змеев, которые летают в воздухе.'] 
['Светофор, который горит красным.'] 
['Черно-белое изображение самолета, летящего в небе.'] 
['Автомобиль едет по улице ночью.'] 
['Светофор, на котором есть несколько изображений.'] 
['Крупный план человека, держащего ножницы.'] 
['Черно-белое изображение птицы, летящей над головой.'] 
['Две фотографии женщин, одетых в купальники.'] 
['Две фотографии женщины, лежащей на кровати.'] 
['Двое мужчин стоят рядом друг с другом и фотографируются.'] 
['Куча плюшевых мишек, сидящих рядом друг с другом.'] 
['Крупный план знака остановки на стене здания.'] 
['Вид на ночное небо с"""

In [28]:
import re
reg = re.compile('[^а-яА-Я ]')
text = reg.sub('', text)

In [33]:
summarizer(text[:500])

[{'summary_text': 'В Чернобельный сидящая в поставить мо крупут н двямых гли-за бы у чи т-о,�е,�е'}]

### IlyaGusev mbart_ru_sum_gazeta 

#### train

In [34]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("IlyaGusev/mbart_ru_sum_gazeta")
model = AutoModelForSeq2SeqLM.from_pretrained("IlyaGusev/mbart_ru_sum_gazeta")

In [35]:
tokenizer.vocab_size

250027

#### Inference

In [36]:
summarizer = pipeline("summarization", model="IlyaGusev/mbart_ru_sum_gazeta")

In [37]:
summarizer(text)

[{'summary_text': 'Стеклянная ваза с нарисованными на ней цветами Стеклянная ваза с нарисованным на ней лицом Стеклянная скульптура с надписью Я не могу видеть изображение выше Чернобелое фото человека сидящего на полу Стеклянная скульптура слона сидящего на столе Стеклянная скульптура слона сидящего на столе Стеклянная ваза с нарисованным на ней лицом Стеклянная ваза стоит на столе Синебелый уличный фонарь и дерево Крупный план часов на стене здания Крупный план цветка на фоне неба Чернобелая фотография человека стоящего на коленях на земле Крупный план уличного знака на фоне голубого неба Крупный план знака остановки на фоне неба Уличная сцена с изображением знака Стоп Чернобелое фото трех человек держащих плюшевых мишек Группа людей стоящих рядом друг с'}]